# Search API

In [1]:
#Few libraries we will using 
#!pip install ujson requests elasticsearch ipywidgets certifi elasticsearch_dsl
#!pip install -U certifi
#!pip install faker
from faker import Faker
import requests
import ujson as json
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from elasticsearch_dsl import Search, DocType, Date, Integer, Keyword, Text
from datetime import datetime
from elasticsearch_dsl.connections import connections
import pandas as pd
from ipywidgets import interact, interactive, fixed, interact_manual, HBox, Dropdown
from IPython.display import clear_output

fake = Faker()
ES_HOST = 'http://34.205.15.150:9200'
INDEX ='newyork'
DOC_TYPE = 'tweet'
es = Elasticsearch(ES_HOST)
print(es)
esurl = ES_HOST+"/"+INDEX+"/"+DOC_TYPE+"/" 
# esurl to be used whenver makeing CURL requests via requests library

<Elasticsearch([{'host': '34.205.15.150', 'port': 9200}])>


```javscript
GET /_search
{
    "query" : {
        "term" : { "user" : "kimchy" }
    }
}
```

In [123]:
doc = {
    "query" : {
        "term" : { "user" : "abc" }
    }
}
res = es.search(index=INDEX, doc_type=DOC_TYPE, body=json.dumps(doc))
print(res['hits']['hits'][0]['_source'])

{'timestamp': 1490605020, 'bids': [459, 66], 'user': 'abc', 'text': 'Reiciendis quis qui nam fugiat nesciunt. Ipsum vero eaque aut vero.', 'age': 3}


In [166]:
## Lets Put 1000 tweets with name ABC
for i in range(0,25):
    doc = {
    'user': 'abc',
    'text': fake.text(),
    'age': int(fake.numerify('##')),
    'bids': '',
    #'bids': [int(fake.numerify('##')), int(fake.numerify('##'))],
    'timestamp': fake.date_time_this_year(),
    }
    res = es.index(index=INDEX, doc_type=DOC_TYPE, id=i, body=json.dumps(doc))
   
    


In [86]:
## Simple Search
doc = {
    "query" : {
        "term" : { "user" : 'abc' }
    }
}
res = es.search(index=INDEX, doc_type=DOC_TYPE, body=json.dumps(doc))
print("--Simple Query--")
print(res['hits']['total'])


## Search with size and starting from
doc = {
    "from" : 0, 
    "size" : 25,
    "query" : {
        "term" : { "user" : "abc" }
    }
}
res = es.search(index=INDEX, doc_type=DOC_TYPE, body=json.dumps(doc))
print("--Search with size and starting from--")
print(len(res['hits']['hits']))

## Search with sorting
doc={
    "from" : 0, 
    "size" : 15,
    "sort" : [
        { "timestamp" : {"order" : "asc"}},
        
    ],
    "query" : {
        "term" : { "user" : "abc" }
    }
}
res = es.search(index=INDEX, doc_type=DOC_TYPE, body=json.dumps(doc))
print("--Search simple sorting--")
for row in res['hits']['hits']:
    time = datetime.datetime.fromtimestamp(
        int(row['_source']['timestamp'])
    ).strftime('%Y-%b-%d')
    print("{user} whose {age} is posted on {time}".format(age = row['_source']['age'],user = row['_source']['user'], time = time))
#print(res['hits']['hits'])



doc={
    "from" : 0, 
    "size" : 15,
    "sort" : [
         { "age" : {"order" : "desc"} },
         {"timestamp" : {"order" : "asc"}}
       
        
    ],
    "query" : {
        "term" : { "user" : "abc" }
    }
}




res = es.search(index=INDEX, doc_type=DOC_TYPE, body=json.dumps(doc))
print("--Search advance sorting--")
for row in res['hits']['hits']:
    time = datetime.datetime.fromtimestamp(
        int(row['_source']['timestamp'])
    ).strftime('%Y-%b-%d')
    print("{user} whose {age} is posted on {time}".format(age = row['_source']['age'],user = row['_source']['user'], time = time))
#print(res['hits']['hits'])



--Simple Query--
999
--Search with size and starting from--
25
--Search simple sorting--
abc whose 1 is posted on 2017-Jan-01
abc whose 94 is posted on 2017-Jan-01
abc whose 83 is posted on 2017-Jan-01
abc whose 20 is posted on 2017-Jan-01
abc whose 70 is posted on 2017-Jan-02
abc whose 32 is posted on 2017-Jan-02
abc whose 29 is posted on 2017-Jan-02
abc whose 43 is posted on 2017-Jan-02
abc whose 68 is posted on 2017-Jan-02
abc whose 2 is posted on 2017-Jan-02
abc whose 80 is posted on 2017-Jan-03
abc whose 4 is posted on 2017-Jan-03
abc whose 75 is posted on 2017-Jan-03
abc whose 38 is posted on 2017-Jan-03
abc whose 29 is posted on 2017-Jan-03
--Search advance sorting--
abc whose 99 is posted on 2017-Jan-03
abc whose 99 is posted on 2017-Jan-14
abc whose 99 is posted on 2017-Jan-31
abc whose 99 is posted on 2017-Feb-03
abc whose 99 is posted on 2017-Feb-06
abc whose 99 is posted on 2017-Apr-06
abc whose 99 is posted on 2017-Apr-22
abc whose 99 is posted on 2017-Apr-22
abc whose 99 

#### Memory Considerationsedit
When sorting, the relevant sorted field values are loaded into memory. This means that per shard, there should be enough memory to contain them. For string based types, the field sorted on should not be analyzed / tokenized. For numeric types, if possible, it is recommended to explicitly set the type to narrower types (like short, integer and float).

In [169]:
doc={
    "from" : 20, 
    "size" : 10,
    "query" : {
      "term" : { "user" : "abc" }
   },
   "sort" : [
      {"bids" : {"missing" : "_first"} },
      {"bids" : {"order" : "asc", "mode" : "avg"}}
   ]
}


res = es.search(index=INDEX, doc_type=DOC_TYPE, body=json.dumps(doc))
print("--Search advance sorting--")
for row in res['hits']['hits']:
    time = datetime.datetime.fromtimestamp(
        int(row['_source']['timestamp'])
    ).strftime('%Y-%b-%d')
    print("{user} whose {age} is posted on {time} has bids {bids}".format(bids = row['_source']['bids'],
                                                          age = row['_source']['age'],user = row['_source']['user'], time = time))
#print(res['hits']['hits'])

--Search advance sorting--
abc whose 48 is posted on 2017-Apr-29 has bids 
abc whose 39 is posted on 2017-Feb-28 has bids 
abc whose 99 is posted on 2017-Apr-13 has bids 
abc whose 58 is posted on 2017-Feb-22 has bids 
abc whose 46 is posted on 2017-Feb-06 has bids 
abc whose 94 is posted on 2017-Feb-26 has bids [0, 4]
abc whose 60 is posted on 2017-Feb-25 has bids [5, 0]
abc whose 90 is posted on 2017-Apr-25 has bids [30, 0]
abc whose 40 is posted on 2017-Feb-22 has bids [0, 43]
abc whose 9 is posted on 2017-Mar-08 has bids [45, 0]


### Source filteringe
Allows to control how the _source field is returned with every hit.

In [ ]:
GET /_search
{
    "_source": [ "obj1.*", "obj2.*" ],
    "query" : {
        "term" : { "user" : "kimchy" }
    }
}

In [174]:
data={
    "_source": [ "bids", "user" ],
    "query" : {
        "term" : { "user" : "abc" }
    }
}
res = es.search(index=INDEX, doc_type=DOC_TYPE,
           body=json.dumps(data))
print(res['hits']['hits'][0]['_source'])

{'user': 'abc', 'bids': [12, 43]}
